# User Story for Calculating ELO Scores of QA Configurations for Ranking 

As a user of the Intelligence Layer (IL), I want to evaluate how well different configurations perform on a QA task with the given input data.
A configuration is a combination of a model with a fixed set of parameters.
In the following, we focus on comparing setups which differ only in the chosen model.

We provide multiple inputs consisting of a longer texts and a questions related to each of those texts, as well as the expected answers.
A Llama-model is used as a grader to decide which answer of two different models is better.
The aggregation of all comparisons results in [ELO](https://en.wikipedia.org/wiki/Elo_rating_system) scores and win rates of the models.

In this notebook, we go through the following steps: First, we create a set of examples of texts with a relevant question for each (Step 0), after which we use the models to generate answers (Step 1). The given answers are then compared against each other and judged by the Llama model (Step 2), which will result in a final ELO ranking and win rate (Step 3). Lastly, we include a new model in the evaluation without having to re-evaluate the previous models against each other, as is typically done in ELO rankings (Step 4).

## Evaluating classification use-cases

Before we can begin, we need to load the Aleph-Alpha access token from the environment and create the client.

In [ ]:
from os import getenv

from aleph_alpha_client import Client
from dotenv import load_dotenv

from intelligence_layer.connectors import LimitedConcurrencyClient
from intelligence_layer.core import Language, Llama3InstructModel, LuminousControlModel
from intelligence_layer.evaluation import (
    AggregatedEvaluation,
    Aggregator,
    Example,
    IncrementalEvaluator,
    InMemoryAggregationRepository,
    InMemoryDatasetRepository,
    InMemoryEvaluationRepository,
    InMemoryRunRepository,
    Matches,
    MatchesAggregationLogic,
    Runner,
)
from intelligence_layer.examples import (
    EloQaEvaluationLogic,
    SingleChunkQa,
    SingleChunkQaInput,
    SingleChunkQaOutput,
)

load_dotenv()

aa_client = Client(getenv("AA_TOKEN"))
limited_concurrency_client = LimitedConcurrencyClient(aa_client)

# Step 0 – Data set

During the four steps of determining the ELO scores, we make use of the following four repositories for managing the intermediate data.

First, we create and store an input dataset into a so-called `dataset_repository`.

The IL will read the input dataset and produce outputs for each model, which will be stored in a `run_repository`.

The result from the previous step can now be evaluated, in this case with an incremental evaluator (`IncrementalEvaluator`), with a QA specific ELO evaluation logic. The evaluation is stored in the `eval_repository`.

Finally, the evaluations are aggregated and stored in the `aggregation_repository`. The aggregation contains the ELO score and winning rate of each model along with additional metadata.

In [ ]:
dataset_repository = InMemoryDatasetRepository()
run_repository = InMemoryRunRepository()
evaluation_repository = InMemoryEvaluationRepository()
aggregation_repository = InMemoryAggregationRepository()

Here, we fill the `dataset_repository` with two `Example`s. Each `Example` contains a text, a question regarding said text, as well as an expected answer.
The newly created dataset in the repository has a unique id, which is stored in the `dataset_id` variable.

In [ ]:
qa_input_text_1 = """Surface micromachining

Surface micromachining builds microstructures by deposition and etching structural layers over a substrate.[1] This is different from Bulk micromachining, in which a silicon substrate wafer is selectively etched to produce structures.

Layers

Generally, polysilicon is used as one of the substrate layers while silicon dioxide is used as a sacrificial layer. The sacrificial layer is removed or etched out to create any necessary void in the thickness direction. Added layers tend to vary in size from 2-5 micrometres. The main advantage of this machining process is the ability to build electronic and mechanical components (functions) on the same substrate. Surface micro-machined components are smaller compared to their bulk micro-machined counterparts.

As the structures are built on top of the substrate and not inside it, the substrate's properties are not as important as in bulk micro-machining. Expensive silicon wafers can be replaced by cheaper substrates, such as glass or plastic. The size of the substrates may be larger than a silicon wafer, and surface micro-machining is used to produce thin-film transistors on large area glass substrates for flat panel displays. This technology can also be used for the manufacture of thin film solar cells, which can be deposited on glass, polyethylene terepthalate substrates or other non-rigid materials.

Fabrication process

Micro-machining starts with a silicon wafer or other substrate upon which new layers are grown. These layers are selectively etched by photo-lithography; either a wet etch involving an acid, or a dry etch involving an ionized gas (or plasma). Dry etching can combine chemical etching with physical etching or ion bombardment. Surface micro-machining involves as many layers as are needed with a different mask (producing a different pattern) on each layer. Modern integrated circuit fabrication uses this technique and can use as many as 100 layers. Micro-machining is a younger technology and usually uses no more than 5 or 6 layers. Surface micro-machining uses developed technology (although sometimes not enough for demanding applications) which is easily repeatable for volume production."""

example_1 = Example(
    input=SingleChunkQaInput(
        chunk=qa_input_text_1,
        question="What is micromachining?",
        language=Language("en"),
    ),
    expected_output="Surface micromachining builds microstructures by deposition and etching structural layers over a substrate. This is different from Bulk micromachining, in which a silicon substrate wafer is selectively etched to produce structures.",
)

qa_input_text_2 = """
Silicon is a chemical element; it has symbol Si and atomic number 14. It is a hard, brittle crystalline solid with a blue-grey metallic luster, and is a non metal and semiconductor. It is a member of group 14 in the periodic table: carbon is above it; and germanium, tin, lead, and flerovium are below it. It is relatively unreactive.

Because of its high chemical affinity for oxygen, it was not until 1823 that Jöns Jakob Berzelius was first able to prepare it and characterize it in pure form. Its oxides form a family of anions known as silicates. Its melting and boiling points of 1414 °C and 3265 °C, respectively, are the second highest among all the metalloids and nonmetals, being surpassed only by boron.[a]

Silicon is the eighth most common element in the universe by mass, but very rarely occurs as the pure element in the Earth's crust. It is widely distributed in space in cosmic dusts, planetoids, and planets as various forms of silicon dioxide (silica) or silicates. More than 90% of the Earth's crust is composed of silicate minerals, making silicon the second most abundant element in the Earth's crust (about 28% by mass), after oxygen.
"""
example_2 = Example(
    input=SingleChunkQaInput(
        chunk=qa_input_text_2, question="What is silicon?", language=Language("en")
    ),
    expected_output="Silicon is a chemical element.",
)

examples = [example_1, example_2]

In [ ]:
dataset_id = dataset_repository.create_dataset(
    examples=examples, dataset_name="My-test-dataset"
).id

In [ ]:
# ensure that we got a valid dataset ID
assert len(dataset_id) > 0, f"The dataset with ID {dataset_id} is empty"

Now that we stored the examples into the `dataset_repository`, we can retrieve them by the `dataset_id`

In [ ]:
for example in dataset_repository.examples(dataset_id, SingleChunkQaInput, str):
    print(example)

# Step 1 - Run Models

Given a `dataset_repository` with examples, we can now generate the output of the models for all examples.
First, we have to define which models we want to use. In this example, we use the _"luminous-base-control"_ model and the _"luminous-supreme-control"_ model.
 
The previously created client is used to create a `Task` for each model. We use a `SingleChunkQa` task, meaning that in each task a model will give an answer to a question regarding a single chunk of text.
These tasks are executed by a `Runner`, using the input dataset via the previously stored `dataset_id`.

Tasks require a `run_repository` to store the output. The generated output is automatically stored when calling `run_dataset` on the `runners`. The output for each model will have a unique _"run id"_.
In general, the output for each model consists of two parts. One part is a collection of example outputs. Each example outputs contains the `run_id`, `example_id`, and a field `output`. In this specific use-case, the `output` field contains the `answer` to the question. The other part is a _"run overview"_ with the run id stored as `id`, the `dataset_id`, and a description of the task, plus other metadata. 

In [ ]:
models = [
    LuminousControlModel(name="luminous-base-control-20240215", client=aa_client),
    LuminousControlModel(name="luminous-supreme-control-20240215", client=aa_client),
]

for model in models:
    runner = Runner[SingleChunkQaInput, SingleChunkQaOutput](
        task=SingleChunkQa(model=model),
        dataset_repository=dataset_repository,
        run_repository=run_repository,
        description=f"QA with model {model.name}",
    )
    runner.run_dataset(dataset_id)

In [ ]:
# ensure that all examples succeeded
for run_overview in run_repository.run_overviews():
    assert (
        run_overview.failed_example_count == 0
    ), f"There are failed runs for run overview ID {run_overview.id}"

The overviews and outputs can be retrieved via the unique run ids:

In [ ]:
print(
    f"Run overview IDs saved in the run repository: {run_repository.run_overview_ids()}\n"
)

for run_overview in run_repository.run_overviews():
    print(run_overview)
    for example_output in run_repository.example_outputs(
        run_overview.id, SingleChunkQaOutput
    ):
        print(example_output)

# Step 2 – Run Evaluation

Now that we have generated the answers of all models for all examples in the `dataset_repository`, the next step is to evaluate those answers.
The evaluation is done by an `Evaluator`. In this notebook we choose an `IncrementalEvaluator` which has the capability to later add additional runs or models without repeating old comparisons, which will come in handy later.

Since we are interested in the ELO score, we use an `EloEvaluationLogic` with our `Evaluator`. This logic compares two outputs against each other and chooses a winner. In order to deem which of the two options is "better" we need to provide a use case specific evaluation logic. In our QA case, this is the `EloQaEvaluationLogic`. We just need to tell the `EloQaEvaluationLogic` which "referee model" it should use to perform the comparison.

In [ ]:
# this should demonstrate that there are no stored evaluations yet in our repository
print(f"IDs of stored evaluations: {evaluation_repository.evaluation_overview_ids()}")

In [ ]:
elo_qa_evaluation_logic = EloQaEvaluationLogic(
    model=Llama3InstructModel(name="llama-3.1-8b-instruct")
)

evaluator = IncrementalEvaluator(
    dataset_repository=dataset_repository,
    run_repository=run_repository,
    evaluation_repository=evaluation_repository,
    description="ELO QA evaluation",  # this description will be used later to query for specific evaluations
    incremental_evaluation_logic=elo_qa_evaluation_logic,
)

In [ ]:
evaluation_overview = evaluator.evaluate_runs(*run_repository.run_overview_ids())

In [ ]:
# check that the evaluation did not crash
failed_evals = evaluation_repository.failed_example_evaluations(
    evaluation_overview.id, Matches
)
assert (
    len(failed_evals) == 0
), f"There are failed evaluations when there should not be:\n {failed_evals}"

# ensure that for each example there are evaluated comparisons
for example_evaluation in evaluation_repository.example_evaluations(
    evaluation_overview.id, Matches
):
    assert isinstance(example_evaluation.result, Matches)
    assert (
        len(example_evaluation.result.comparison_evaluations) > 0
    ), f"There are no matches (comparisons) for example ID {example_evaluation.example_id}"

The evaluation results can be retrieved via their unique ids:

In [ ]:
for evaluation_overview in evaluation_repository.evaluation_overviews():
    print(evaluation_overview)

# Step 3 – Run Aggregation

Finally, all individual evaluations are aggregated into metrics for each model - here, an ELO score and a win rate.
The `MatchesAggregationLogic` defines how the evaluations should be aggregated for the ELO use case and can be customized.

In [ ]:
# this should demonstrate that there are no stored aggregated evaluations yet in our repository
print(
    f"IDs of stored aggregated evaluations: {aggregation_repository.aggregation_overview_ids()}"
)

In [ ]:
aggregator = Aggregator(
    evaluation_repository=evaluation_repository,
    aggregation_repository=aggregation_repository,
    description="ELO QA aggregation",
    aggregation_logic=MatchesAggregationLogic(),
)

aggregated_evaluation = aggregator.aggregate_evaluation(evaluation_overview.id)

In [ ]:
# ensure that there are no failed (aggregated) evaluations
assert (
    aggregated_evaluation.crashed_during_evaluation_count == 0
), f"There are crashed evaluations for aggregated evaluation ID {aggregated_evaluation.id}"
assert (
    aggregated_evaluation.failed_evaluation_count == 0
), f"There are failed evaluations for aggregated evaluation ID {aggregated_evaluation.id}"
# ensure that the result contains ELO scores
assert hasattr(
    aggregated_evaluation.statistics, "scores"
), f"There are no scores for aggregated evaluation ID {aggregated_evaluation.id}"

We can get an overview of each aggregation from the aggregation repository as follows. Note that we need to provide the type of the aggregation to enable the deserialization. The given `statistics` field of the evaluation result contains only the aggregated metrics for each model. 

In [ ]:
for aggregation_overview in aggregation_repository.aggregation_overviews(
    AggregatedEvaluation
):
    print(aggregation_overview)

# Step 4 - Addition of New Models

Now let us consider the case where we want to add new models to our existing evaluation.
Since the comparison of answers is rather time-consuming, we want to avoid recalculating the evaluation for the previous models, and just compare the new models to the old ones. This is why we used the `IncrementalEvaluator` to begin with.

For this example, we first define the new models and generate their answers.

In [ ]:
newly_added_models = [
    LuminousControlModel(name="luminous-base-control-20230501", client=aa_client),
    LuminousControlModel(name="luminous-supreme-control-20230501", client=aa_client),
]

for model in newly_added_models:
    runner = Runner[
        SingleChunkQaInput, SingleChunkQaOutput
    ](
        task=SingleChunkQa(model),
        dataset_repository=dataset_repository,
        run_repository=run_repository,
        description=f"New QA with model {model.name}",  # used to query for new runs only later in the code
    )
    runner.run_dataset(dataset_id)

In [ ]:
# ensure that all examples succeeded
for run_overview in run_repository.run_overviews():
    assert (
        run_overview.failed_example_count == 0
    ), f"There are failed runs for run overview ID {run_overview.id}"

In [ ]:
for run_overview in run_repository.run_overviews():
    # skip runs done for previous models
    if not run_overview.description.startswith("New"):
        continue
    # print runs for the added models
    print(run_overview)

Thanks to the `IncrementalEvaluator`, we can now easily extend our existing evaluation with the comparisons of new model runs against the previous runs, without re-running the previous comparisons. To this end, we use the same evaluator instance as for our first evaluation, but use the `evaluate_additional_runs` method, which takes a list of previous evaluation_overview IDs and uses them to filter the resulting comparisons. In this case, only comparisons of new pairings will be performed.

In [ ]:
new_evaluation_overview = evaluator.evaluate_additional_runs(
    *run_repository.run_overview_ids(), previous_evaluation_ids=[evaluation_overview.id]
)

In [ ]:
# ensure that for each example there are evaluated comparisons
for example_evaluation in evaluation_repository.example_evaluations(
    new_evaluation_overview.id, Matches
):
    assert isinstance(example_evaluation.result, Matches)
    assert (
        len(example_evaluation.result.comparison_evaluations) > 0
    ), f"There are no matches (comparisons) for example ID {example_evaluation.example_id}"

In addition to the previous `evaluation_overview`, we now also have the newly generated `new_evaluation_overview` which includes our new model.
Finally, the aggregated evaluation of all models is calculated by passing in the evaluation ids of both evaluations into `aggregate_evaluation`. By doing so, the previously calculated ELO scores will be updated with the comparisons to the new models' answers.

In [ ]:
# get the IDs of all the evaluation overviews which we created for the QA task
evaluation_overview_ids = [
    evaluation_overview.id
    for evaluation_overview in evaluation_repository.evaluation_overviews()
    if evaluation_overview.description.find("QA")
]
print(f"Evaluation overviews to aggregate: {evaluation_overview_ids}")

In [ ]:
# run the aggregation
aggregated_evaluation_with_new_model = aggregator.aggregate_evaluation(
    *evaluation_overview_ids
)

In [ ]:
# ensure that there are no failed (aggregated) evaluations
assert (
    aggregated_evaluation_with_new_model.crashed_during_evaluation_count == 0
), f"There are crashed evaluations for aggregated evaluation ID {aggregated_evaluation.id}"
assert (
    aggregated_evaluation_with_new_model.failed_evaluation_count == 0
), f"There are failed evaluations for aggregated evaluation ID {aggregated_evaluation.id}"
# ensure that we result contains ELO scores
assert hasattr(
    aggregated_evaluation_with_new_model.statistics, "scores"
), f"There are no scores for aggregated evaluation ID {aggregated_evaluation.id}"

A look at the new aggregated evaluation shows that the runs of the new models have been added to the evaluation:

In [ ]:
print(aggregated_evaluation_with_new_model)